In [3]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## _Sorting out cells that made it to landfall (used the globe.is_land() library)_

In [4]:
track = pd.read_csv('../saved-files/threshold-5/final-tracks-threshold-5.csv')
track

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0         4   28  201.219736  300.689233   11                5      423   
1         5   38  202.273236  301.660422   35                5      527   
2         6   37  203.303419  302.324596   49                5      627   
3         4   37  230.460885  211.799724  187                5      432   
4         5   44  231.873681  213.475405  183                5      533   
...     ...  ...         ...         ...  ...              ...      ...   
6155    299   27  271.955203  288.680268  159                5    23039   
6156    300   30  272.843216  289.783693  142                5    23077   
6157    298   34  296.371715  217.621195   30                5    23006   
6158    299   30  296.984193  220.121651   26                5    23042   
6159    300   34  298.121546  222.140438   35                5    23081   

                     time              timestr  projection_y_coordinate  \
0     2020-03-13 04:20:00  2020-03-13 04:20:00            -2.261220e+06   
1     2020-03-13 04:25:00  2020-03-13 04:25:00            -2.262273e+06   
2     2020-03-13 04:30:00  2020-03-13 04:30:00            -2.263303e+06   
3     2020-03-13 04:20:00  2020-03-13 04:20:00            -2.290461e+06   
4     2020-03-13 04:25:00  2020-03-13 04:25:00            -2.291874e+06   
...                   ...                  ...                      ...   
6155  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.331955e+06   
6156  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.332843e+06   
6157  2020-03-14 04:50:00  2020-03-14 04:50:00            -2.356372e+06   
6158  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.356984e+06   
6159  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.358122e+06   

      projection_x_coordinate   latitude  longitude   cell        time_cell  \
0               217689.233099  69.544704  15.498959    416  0 days 00:00:00   
1               218660.421950  69.534304  15.520784    416  0 days 00:05:00   
2               219324.595821  69.524381  15.534945    416  0 days 00:10:00   
3               128799.724015  69.340584  13.218530    425  0 days 00:00:00   
4               130475.405470  69.326868  13.258308    425  0 days 00:05:00   
...                       ...        ...        ...    ...              ...   
6155            205680.267926  68.912816  15.040489  16225  0 days 00:05:00   
6156            206783.692918  68.903864  15.065471  16225  0 days 00:10:00   
6157            134621.195276  68.737655  13.269794  16228  0 days 00:00:00   
6158            137121.651293  68.730769  13.329528  16228  0 days 00:05:00   
6159            139140.437662  68.719331  13.376810  16228  0 days 00:10:00   

        distance   mean dBZ    max dBZ  area($km^2$)  
0      39.132921  12.845599  38.092506     50.741650  
1      37.704661  13.138674  36.584485     49.744965  
2      36.472873  13.454442  38.092506     31.837476  
3     110.156057  12.845599  38.092506    360.167853  
4     108.487906  13.138674  36.584485    362.160182  
...          ...        ...        ...           ...  
6155   52.916488  15.240493  41.116443    452.731337  
6156   52.910819  14.702386  40.106802    453.726574  
6157  122.630657  15.301029  39.603226    161.199380  
6158  120.888777  15.240493  41.116443     75.621784  
6159  119.862226  14.702386  40.106802    157.215939  

[6160 rows x 19 columns]

In [5]:
# tracks_including_area = pd.read_csv('./saved-files/final-tracks.csv')
# tracks_including_area.head()

In [6]:
# area = tracks_including_area['area($km^2$)']
# # add area pandas series to track dataframe
# track['area'] = area

In [7]:
track.head()   

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0      4   28  201.219736  300.689233   11                5      423   
1      5   38  202.273236  301.660422   35                5      527   
2      6   37  203.303419  302.324596   49                5      627   
3      4   37  230.460885  211.799724  187                5      432   
4      5   44  231.873681  213.475405  183                5      533   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.261220e+06   
1  2020-03-13 04:25:00  2020-03-13 04:25:00            -2.262273e+06   
2  2020-03-13 04:30:00  2020-03-13 04:30:00            -2.263303e+06   
3  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.290461e+06   
4  2020-03-13 04:25:00  2020-03-13 04:25:00            -2.291874e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            217689.233099  69.544704  15.498959   416  0 days 00:00:00   
1            218660.421950  69.534304  15.520784   416  0 days 00:05:00   
2            219324.595821  69.524381  15.534945   416  0 days 00:10:00   
3            128799.724015  69.340584  13.218530   425  0 days 00:00:00   
4            130475.405470  69.326868  13.258308   425  0 days 00:05:00   

     distance   mean dBZ    max dBZ  area($km^2$)  
0   39.132921  12.845599  38.092506     50.741650  
1   37.704661  13.138674  36.584485     49.744965  
2   36.472873  13.454442  38.092506     31.837476  
3  110.156057  12.845599  38.092506    360.167853  
4  108.487906  13.138674  36.584485    362.160182

In [8]:
%%time
cell_landfall = []
for i, j in enumerate(track['cell']):
#     # # Check if a point is on land:
    lat = track['latitude'][i]
    lon = track['longitude'][i]
    is_on_land = globe.is_land(lat, lon)
    if is_on_land:
        # if ((lat >= 69.0) & (lon >= 12.0)):
            # print(f"lat={lat}, lon={lon} is on land: {is_on_land} -- cell_id={j}")
        cell_landfall.append(j)

CPU times: user 527 ms, sys: 0 ns, total: 527 ms
Wall time: 550 ms


In [9]:
%%time
cells_landfall=np.unique(cell_landfall)
cells_landfall

CPU times: user 102 µs, sys: 0 ns, total: 102 µs
Wall time: 104 µs


array([ 2559,  2780,  2895,  3020,  3143,  3294,  3510,  3661,  3719,
        3734,  3737,  3908,  3961,  4130,  4193,  4402,  4473,  5360,
        5385,  5400,  5519,  5584,  5653,  5693,  5902,  5965,  5981,
        6090,  6523,  6728,  6941,  6989,  7123,  7144,  7287,  7289,
        7292,  7542,  7891,  7901,  8069,  8103,  8146,  8152,  8172,
        8251,  8285,  8405,  8452,  8523,  8574,  8626,  8700,  8816,
        8883,  8890,  8984,  9257,  9363,  9508,  9558,  9669,  9686,
        9694,  9766,  9919,  9920,  9926,  9983, 10178, 10415, 10689,
       10804, 10892, 11008, 11214, 11259, 11293, 11343, 11365, 11366,
       11407, 11517, 11557, 11610, 11618, 11651, 11775, 11829, 11917,
       11926, 12013, 12048, 12148, 12200, 12413, 12495, 12560, 12661,
       12666, 12694, 12835, 12888, 12891, 12960, 13009, 13028, 13093,
       13175, 13332, 13347, 13456, 13563, 13566, 13570, 13684, 13696,
       13705, 13845, 13908, 14213, 14317, 14451, 14515, 14526, 14527,
       14623, 14663,

## _Algorithm to track cells back in time that made it to landfall_

In [10]:
%%time
combined_cells_landfalls = []
for i, cell_ids in enumerate(np.unique(cells_landfall)): # looped through tracks found with AMF1 site
    if cell_ids in track['cell'].values: # checking if those cells(tracks) are within the original tracked dataframe
        # print(f'Cell {cell_ids} present: made it to landfall 😀')
        track_c = track[track['cell'] == cell_ids] # display it if it is within the tracked dataframe
        combined_cells_landfalls.append(track_c) # append the results you get a new dataFrame calld combined_tracks_AMF1
    else:
        print('Cells not present 🥲')
        
# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
combined_cells_landfall_present = pd.concat(combined_cells_landfalls)
combined_cells_landfall_present = combined_cells_landfall_present.reset_index(drop=True)
combined_cells_landfall_present['landfall'] = globe.is_land(combined_cells_landfall_present['latitude'], combined_cells_landfall_present['longitude'])

CPU times: user 51.6 ms, sys: 0 ns, total: 51.6 ms
Wall time: 51.8 ms


In [11]:
combined_cells_landfall_present.head()

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0     30   81  367.164557  350.000000    2                5     3214   
1     31   93  369.234634  351.175654    5                5     3314   
2     32  101  371.445794  352.115530    8                5     3421   
3     33   95  373.250797  352.628364    5                5     3524   
4     33   89  359.592230  312.428919   12                5     3518   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 06:30:00  2020-03-13 06:30:00            -2.427165e+06   
1  2020-03-13 06:35:00  2020-03-13 06:35:00            -2.429235e+06   
2  2020-03-13 06:40:00  2020-03-13 06:40:00            -2.431446e+06   
3  2020-03-13 06:45:00  2020-03-13 06:45:00            -2.433251e+06   
4  2020-03-13 06:45:00  2020-03-13 06:45:00            -2.419592e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            267000.000000  67.992929  16.277576  2559  0 days 00:00:00   
1            268175.653727  67.972954  16.299665  2559  0 days 00:05:00   
2            269115.529672  67.951937  16.315863  2559  0 days 00:10:00   
3            269628.364433  67.935033  16.323145  2559  0 days 00:15:00   
4            229428.918997  68.096681  15.416666  2780  0 days 00:00:00   

     distance   mean dBZ    max dBZ  area($km^2$)  landfall  
0  139.399707  14.490822  40.610380     13.930986      True  
1  141.690055  14.672790  39.605794     19.905209      True  
2  144.078219  14.937969  39.099655           NaN      True  
3  145.979154  14.874514  39.605794     14.927725      True  
4  129.596310  14.874514  39.605794     16.918205      True

In [12]:
len(combined_cells_landfall_present)

590

## _track up to the time the cell first hit landfall (after it hits the first landfall, remove all the subsequent cells hitting landfall, we are not interested in it)_

In [13]:
%%time
cells_to_first_landfall = []
for i, cell_idx in enumerate(np.unique(cells_landfall)):
    df = combined_cells_landfall_present[combined_cells_landfall_present['cell'] == cell_idx]
    # # find the index of the first occurrence of True
    idx = df.loc[df['landfall']].index[0]
    # slice the DataFrame to keep only the rows up to and including that index
    df = df.loc[:idx]
    # set all subsequent occurrences of True to False
    df.loc[df['landfall'].shift(-1) == True, 'landfall'] = False
    # print(df)
    cells_to_first_landfall.append(df)
first_landfall = pd.concat(cells_to_first_landfall)

CPU times: user 134 ms, sys: 0 ns, total: 134 ms
Wall time: 153 ms


## _calculate relative time --- relative time = time - time_landfall_

In [14]:
%%time
cells_relative_time = []
for index, cell_id in enumerate(np.unique(first_landfall['cell'])):
    if cell_id:
        # print(cell_id)
        cells = first_landfall[first_landfall['cell'] == cell_id] # get all the cell_ids
    # calculate for the relative time ----- relative time = time - time_landfall
        all_cells_times = cells['timestr']; all_cells_times = pd.to_datetime(all_cells_times.values) # convert timestr to datetime
        landfall_time_each_cell = all_cells_times[::-1][0]
        # relative_time = np.absolute(all_cells_times - landfall_time_each_cell) # relative time
        relative_time = np.subtract(landfall_time_each_cell, all_cells_times)
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time.append(relative_time)

CPU times: user 110 ms, sys: 0 ns, total: 110 ms
Wall time: 111 ms


In [15]:
cells_relative_time[:20]

[Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64')]

In [16]:
# cells_relative_time
cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
# first_landfall['relative_time'] = cells_relative_times
# cells_relative_times.values
first_landfall['relative_time'] = cells_relative_times.values

In [17]:
len(first_landfall)

256

In [18]:
first_landfall.to_csv('../saved-files/threshold-5/all-cells-first_landfall-threshold-5.csv', index=False)

In [19]:
# final = first_landfall[['cell', 'latitude', 'longitude', 'distance', 'max dBZ','mean dBZ', 'area' ,'timestr', 'landfall']]
# final

# END 